## Generate openSMILE Embeddings

### Install dependencies

In [1]:
!pip install opensmile -q

In [2]:
!pip install boxsdk -q
!pip install dotenv -q

In [3]:
!pip install numpy -q
!pip install torch -q

### Box Authorization

In [ ]:
from dotenv import load_dotenv
from boxsdk import OAuth2, Client
import os

load_dotenv()

auth = OAuth2(
    client_id=os.getenv('BOX_CLIENT_ID'),
    client_secret=os.getenv('BOX_CLIENT_SECRET'),
    access_token=os.getenv('BOX_ACCESS_TOKEN'),
)
client = Client(auth)

### Stream Data

In [ ]:
from pathlib import Path
from io import BytesIO
import soundfile as sf

def stream_data(filename: str, client: Client, folder_id: int):
    """
    file_name: name of a .wav file (including extension) located somewhere in the Box folder identified by folder_id
    client: authenticated boxsdk.Client
    folder_id: Box folder ID (root for the search scope)

    Uses the Box Search API scoped to the ancestor folder to find the file by name,
    downloads the file bytes and decodes the WAV into a waveform and sampling rate.

    Returns: (waveform (numpy array), sampling_rate)
    Raises FileNotFoundError if the file cannot be found.

    """
    # use Box search scoped to the ancestor folder to find candidate files
    results = client.search().query(query=filename, file_extensions=['wav'], ancestor_folder_ids=[str(folder_id)], limit=100)

    candidate = None
    for item in results:
        # ensure it's a file and the name matches exactly
        if getattr(item, 'type', None) == 'file' and getattr(item, 'name', None) == filename:
            candidate = item
            break

    if not candidate:
        raise FileNotFoundError(f'File not found: {filename}')

    # download bytes (for large files prefer download_to with a stream to disk)
    file_bytes = client.file(candidate.id).content()
    bio = BytesIO(file_bytes)

    # decode waveform
    waveform, sr = sf.read(bio)
    return waveform, sr

### openSMILE Feature Extractor

In [ ]:
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,  # ComParE_2016 or emobase or eGeMAPSv02
    feature_level=opensmile.FeatureLevel.Functionals,
)

def extract_features(waveform, sampling_rate):
    """
    waveform: waveform of a .wav file
    sampling_rate: sampling rate of a .wav file

    Returns extracted features using the openSMILE feature extractor

    """
    return smile.process_signal(waveform, sampling_rate)

### Save Embeddings

In [ ]:
import os
import numpy as np
from pathlib import Path
from typing import Union

def save_embedding(features: np.ndarray, out_path: Path) -> None:
    """
    features: 1-D array (single row) or 2-D array (one or more rows)
    out_path: Path to a .npy file that stores a 2-D array (rows x features)

    Writes the features to out_path. If out_path already exists, appends the new features

    """
    out_path.parent.mkdir(parents=True, exist_ok=True)

    # Ensure features is a numpy array
    feat = np.asarray(features)
    if feat.ndim == 1:
        feat = feat[np.newaxis, :]
    elif feat.ndim != 2:
        raise ValueError("`features` must be 1-D or 2-D numpy array")

    if out_path.exists():
        existing = np.load(out_path)
        # Normalize existing to 2-D
        if existing.ndim == 1:
            existing = existing[np.newaxis, :]

        # Check column consistency
        if existing.shape[1] != feat.shape[1]:
            raise ValueError(f"Column mismatch: existing has {existing.shape[1]} cols but new features have {feat.shape[1]} cols")

        combined = np.vstack([existing, feat])
    else:
        combined = feat

    # Atomic write: save to temp then replace
    tmp_path = out_path.with_suffix(out_path.suffix + ".tmp")
    np.save(tmp_path, combined)
    os.replace(tmp_path, out_path)

### Extract and Save Embeddings

In [ ]:
import csv

def extract_and_save_embeddings(csv_path: Path, out_path: Path, client: Client, folder_id: int) -> None:
    with open(csv_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            filename = row['id'] + '.wav'
            try:
                waveform, sr = stream_data(filename, client, folder_id)
                features = extract_features(waveform, sr)
                save_embedding(features, out_path)
                print(f"Processed and saved features for {filename}")
            except FileNotFoundError:
                print(f"File not found: {filename}, skipping.")

#### Paths

In [ ]:
DATA_DIR = Path('../../data/iemocap_4way_data')
TRAIN_PATH = DATA_DIR / 'train_4way_with_minus_one.csv'
VAL_PATH = DATA_DIR / 'val_4way_with_minus_one.csv'
TEST_PATH = DATA_DIR / 'test_4way_with_minus_one.csv'

SAVE_DIR = Path('../../data/embeddings/4way_opensmile/eGeMAPSv02')
SAVE_TRAIN_PATH = SAVE_DIR / 'train.npy'
SAVE_VAL_PATH = SAVE_DIR / 'val.npy'
SAVE_TEST_PATH = SAVE_DIR / 'test.npy'

FOLDER_ID = 279758036893

#### Extract and Save Training Embeddings

In [ ]:
extract_and_save_embeddings(TRAIN_PATH, SAVE_TRAIN_PATH, client, FOLDER_ID)

#### Extract and Save Validation Embeddings

In [ ]:
extract_and_save_embeddings(TRAIN_PATH, SAVE_TRAIN_PATH, client, FOLDER_ID)

#### Extract and Save Testing Embeddings

In [ ]:
extract_and_save_embeddings(TRAIN_PATH, SAVE_TRAIN_PATH, client, FOLDER_ID)